### 6.1.1 不变性
想象一下，假设我们想从一张图片中找到某个物体。 合理的假设是：无论哪种方法找到这个物体，都应该和物体的位置无关。 理想情况下，我们的系统应该能够利用常识：猪通常不在天上飞，飞机通常不在水里游泳。 但是，如果一只猪出现在图片顶部，我们还是应该认出它。 我们可以从儿童游戏”沃尔多在哪里”（ 图6.1.1）中得到灵感： 在这个游戏中包含了许多充斥着活动的混乱场景，而沃尔多通常潜伏在一些不太可能的位置，读者的目标就是找出他。 尽管沃尔多的装扮很有特点，但是在眼花缭乱的场景中找到他也如大海捞针。 然而沃尔多的样子并不取决于他潜藏的地方，因此我们可以使用一个“沃尔多检测器”扫描图像。 该检测器将图像分割成多个区域，并为每个区域包含沃尔多的可能性打分。 卷积神经网络正是将空间不变性（spatial invariance）的这一概念系统化，从而基于这个模型使用较少的参数来学习有用的表示。

### 6.1.2 多层感知机的限制

首先，多层感知机的输入是二维图像 $\mathbf{X}$，其隐藏表示 $\mathbf{H}$ 在数学上是一个矩阵，在代码中表示为二维张量。 其中 $\mathbf{X}$ 和 $\mathbf{H}$ 具有相同的形状。 为了方便理解，我们可以认为，无论是输入还是隐藏表示都拥有空间结构。使用 $\mathbf{X}_{i,j}$ 和 $\mathbf{H}_{i,j}$ 分别表示输入图像和隐藏表示中位置 $(i, j)$ 处的像素。 为了使每个隐藏神经元都能接收到每个输入像素的信息，我们将参数从权重矩阵（如同我们先前在多层感知机中所做的那样）替换为四阶权重张量 $\mathsf{W}$。假设 $\mathbf{U}$ 包含偏置参数，我们可以将全连接层形式化地表示为

$$
\begin{aligned}
\left[\mathbf{H}\right]_{i, j} &= [\mathbf{U}]_{i, j} + \sum_k \sum_l[\mathsf{W}]_{i, j, k, l}  [\mathbf{X}]_{k, l}\\
&=  [\mathbf{U}]_{i, j} + \sum_a \sum_b [\mathsf{V}]_{i, j, a, b}  [\mathbf{X}]_{i+a, j+b}.
\end{aligned}
$$

其中，从 $\mathsf{W}$ 到 $\mathsf{V}$ 的转换只是形式上的转换，因为在这两个四阶张量的元素之间存在一一对应的关系。 我们只需重新索引下标 $(k, l)$，使 $k=i+a$、$l=j+b$，由此可得 $[\mathbf{V}]_{i, j, a, b} = [\mathsf{W}]_{i, j, k, l}$。 索引 $a$ 和 $b$ 通过在正偏移和负偏移之间移动覆盖了整个图像。 对于隐藏表示中任意给定位置 $(i, j)$ 处的像素值 $[\mathbf{H}]_{i, j}$，可以通过在 $\mathbf{X}$ 中以 $(i, j)$ 为中心对像素进行加权求和得到，加权使用的权重为 $[\mathsf{V}]_{i, j, a, b}$。

### 6.1.3 卷积

在进一步讨论之前，我们先简要回顾一下为什么上面的操作被称为卷积。在数学中，两个函数（比如 $f, g: \mathbb{R}^d \to \mathbb{R}$）之间的“卷积”被定义为

$$
(f * g)(\mathbf{x}) = \int f(\mathbf{z}) g(\mathbf{x}-\mathbf{z}) d\mathbf{z}.
$$
(6.1.4)

也就是说，卷积是当把一个函数“翻转”并移位 $\mathbf{x}$ 时，测量 $f$ 和 $g$ 之间的重叠。当为离散对象时，积分就变成求和。例如，对于由索引为 $i$ 的、平方可和的、无限维向量集合中抽取的向量，我们得到以下定义：

$$
(f * g)(i) = \sum_a f(a) g(i-a).
$$
(6.1.5)

对于二维张量，则为 $f$ 的索引 $(a, b)$ 和 $g$ 的索引 $(i-a, j-b)$ 上的对应加和：

$$
(f * g)(i, j) = \sum_{a, b} f(a, b) g(i-a, j-b).
$$
(6.1.6)

这看起来类似于 (6.1.3)，但有一个主要区别：这里不是使用 $(i+a, j+b)$，而是使用差值。然而，这种区别是表面的，因为我们总是可以匹配 (6.1.3) 和 (6.1.6) 之间的符号。我们在 (6.1.3) 中的原始定义更正确地描述了互相关（cross-correlation），这个问题将在下一节中讨论。


1. 假设卷积层 (6.1.3) 覆盖的局部区域 $\Delta = 0$。在这种情况下，证明卷积内核为每组通道独立地实现一个全连接层。
2. 为什么平移不变性可能也不是好主意呢？
3. 当从图像边界像素获取隐藏表示时，我们需要思考哪些问题？
4. 描述一个类似的音频卷积层的架构。
5. 卷积层也适合于文本数据吗？为什么？
6. 证明在 (6.1.6) 中，$f * g = g * f$。

__解答：__

1. 当卷积层覆盖的局部区域 $\Delta = 0$ 时，卷积内核的大小为 $1 \times 1$，这意味着每个输出像素仅依赖于对应输入像素的值。因此，对于每组通道，卷积操作等价于一个全连接层，其中每个输入像素与输出像素之间存在一个独立的权重。

2. 平移不变性可能不是好主意的原因是，有些任务需要考虑输入数据的绝对位置。例如，在图像分类中，物体的位置可能对分类结果有重要影响。如果忽略了位置信息，可能会导致分类错误。

3. 当从图像边界像素获取隐藏表示时，我们需要考虑边界效应。由于卷积操作需要访问邻近像素，边界像素的邻近像素可能不存在。这可以通过填充（padding）技术来解决，即在图像边界添加额外的像素，以确保边界像素也有足够的邻近像素进行卷积操作。

4. 类似的音频卷积层架构可以使用一维卷积（1D Convolution）来处理音频信号。音频信号通常表示为一维时间序列数据，因此可以使用一维卷积核在时间轴上滑动，以提取局部特征。可以堆叠多个一维卷积层，以捕捉更高层次的特征。

5. 卷积层也适合于文本数据，因为文本数据可以表示为一维序列（例如单词或字符序列）。通过使用一维卷积层，可以在文本序列上滑动卷积核，以提取局部特征（例如n-grams）。这种方法在自然语言处理任务中，如文本分类和情感分析中，已经取得了成功。

6. 在 (6.1.6) 中，$f * g = g * f$ 可以通过交换求和的顺序来证明。具体来说：

$$
(f * g)(i, j) = \sum_{a, b} f(a, b) g(i-a, j-b)
$$

交换 $f$ 和 $g$ 的位置：

$$
(g * f)(i, j) = \sum_{a, b} g(a, b) f(i-a, j-b)
$$

通过重新命名求和索引，可以看出这两个表达式是相等的。因此，$f * g = g * f$。